## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 14s 0us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
print(y_train.shape)
one_hot=OneHotEncoder()
# https://stackoverflow.com/questions/52846604/using-toarray-with-onehotencoding-during-data-preprocessing
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()
print(y_train.shape)

(50000, 1)
(50000, 10)


In [5]:

classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(filters = 32 , kernel_size=(3 , 3) , strides= (1 , 1) , padding='same' , activation='relu' , input_shape = (32 , 32 , 3)))
classifier.add(BatchNormalization(momentum=0.99 , epsilon=0.001))

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters = 32 , kernel_size=(3 , 3) , strides= (1 , 1) , padding='same' , activation='relu'))
classifier.add(BatchNormalization(momentum=0.99 , epsilon=0.001))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100 , activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=128,epochs=20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/20
50000/50000 [==============================] - 6s 112us/step - loss: 1.2539 - acc: 0.5656
Epoch 2/20
50000/50000 [==============================] - 5s 100us/step - loss: 0.7859 - acc: 0.7250
Epoch 3/20
50000/50000 [==============================] - 5s 100us/step - loss: 0.5457 - acc: 0.8099
Epoch 4/20
50000/50000 [==============================] - 5s 99us/step - loss: 0.3586 - acc: 0.8757
Epoch 5/20
50000/50000 [==============================] - 5s 101us/step - loss: 0.2237 - acc: 0.9236
Epoch 6/20
50000/50000 [==============================] - 5s 100us/step - loss: 0.1493 - acc: 0.9511
Epoch 7/20
50000/50000 [==============================] - 5s 100us/step - loss: 0.1128 - acc: 0.9614
Epoch 8/20
50000/50000 [==============================] - 5s 100us/step - loss: 0.1034 - acc: 0.9641
Epoch 9/20
50000/50000 [==============================] - 5s 99us/step - loss: 0.1033 - acc: 0.9653
Epoch 10/20
50000/50000 [==============================] - 5s 99us/step - loss: 0.0876 - acc:

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [9]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict_classes(input_example)

array([4])

In [12]:
result = classifier.evaluate(x_test , y_test)
print(result[1])

10000/10000 [==============================] - 1s 106us/step
0.6586
